In [290]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [291]:
df = pd.read_csv('kagglex-cohort4/train.csv')

In [292]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36183 entries, 0 to 36182
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            36183 non-null  int64 
 1   brand         36183 non-null  object
 2   model         36183 non-null  object
 3   model_year    36183 non-null  int64 
 4   milage        36183 non-null  int64 
 5   fuel_type     36183 non-null  object
 6   engine        36183 non-null  object
 7   transmission  36183 non-null  object
 8   ext_col       36183 non-null  object
 9   int_col       36183 non-null  object
 10  accident      36183 non-null  object
 11  clean_title   36183 non-null  object
dtypes: int64(3), object(9)
memory usage: 3.3+ MB


In [267]:
threshold = 3

z_scores = df.select_dtypes(include=['number']).apply(lambda x: np.abs((x - x.mean()) / x.std()))

df = df[(z_scores < threshold).all(axis=1)]

In [293]:
import pandas as pd

data = {
    'Brand': ['Ford', 'BMW', 'Jaguar', 'Pontiac', 'Acura', 'Audi', 'GMC',
              'Maserati', 'Chevrolet', 'Porsche', 'Mercedes-Benz', 'Tesla',
              'Lexus', 'Kia', 'Lincoln', 'Dodge', 'Volkswagen', 'Land',
              'Cadillac', 'Mazda', 'RAM', 'Subaru', 'Hyundai', 'MINI', 'Jeep',
              'Honda', 'Hummer', 'Nissan', 'Toyota', 'Volvo', 'Genesis',
              'Mitsubishi', 'Buick', 'INFINITI', 'McLaren', 'Scion',
              'Lamborghini', 'Bentley', 'Suzuki', 'Ferrari', 'Alfa',
              'Rolls-Royce', 'Chrysler', 'Aston', 'Rivian', 'Lotus', 'Saturn',
              'Lucid', 'Mercury', 'Maybach', 'FIAT', 'Plymouth', 'Bugatti'],
    'Country': ['United States', 'Germany', 'United Kingdom', 'United States',
                'Japan', 'Germany', 'United States', 'Italy', 'United States',
                'Germany', 'Germany', 'United States', 'Japan', 'South Korea',
                'United States', 'United States', 'Germany', 'United Kingdom',
                'United States', 'Japan', 'United States', 'Japan', 'South Korea',
                'United Kingdom', 'United States', 'Japan', 'United States',
                'Japan', 'Japan', 'Sweden', 'South Korea', 'Japan', 'United States',
                'Japan', 'United Kingdom', 'United States', 'Italy', 'United Kingdom',
                'Japan', 'Italy', 'Italy', 'United Kingdom', 'United States',
                'United Kingdom', 'United States', 'United Kingdom', 'United States',
                'United States', 'United States', 'Germany', 'Italy', 'United States',
                'France']
}


In [294]:
brand_df = pd.DataFrame(data)

In [295]:
brand_country_map = dict(zip(brand_df['Brand'], brand_df['Country']))
df['brand_country'] = df['brand'].map(brand_country_map)

In [296]:
europe = ['Germany', 'Italy', 'Sweden', 'United Kingdom', 'Slovakia']
asia = ['Japan', 'South Korea']
us = ['United States']

df.loc[df['brand_country'].isin(europe), 'brand_continent'] = 'Europe'
df.loc[df['brand_country'].isin(asia), 'brand_continent'] = 'Asia'
df.loc[df['brand_country'].isin(us), 'brand_continent'] = 'American'

In [297]:
df['fuel'] = df['fuel_type']
df.loc[df['fuel'].str.contains('Hybrid'), 'fuel'] = 'Hybrid'
df.loc[df['fuel'].str.contains('–'), 'fuel'] = 'Other'
df.loc[df['fuel'] == 'not supported', 'fuel'] = 'Other'

In [298]:
df['engine'].str.split(' ',expand = True)

,0,1,2,3,4,5,6,7,8
0,302.0HP,3.5L,V6,Cylinder,Engine,Gasoline,Fuel,None,None
1,275.0HP,3.5L,V6,Cylinder,Engine,Gasoline,Fuel,None,None
2,241.0HP,2.0L,4,Cylinder,Engine,Gasoline,Fuel,None,None
3,518.0HP,5.0L,8,Cylinder,Engine,Gasoline,Fuel,None,None
4,335.0HP,3.0L,Straight,6,Cylinder,Engine,Gasoline,Fuel,None
...,...,...,...,...,...,...,...,...,...
36178,420.0HP,6.2L,8,Cylinder,Engine,Gasoline,Fuel,None,None
36179,295.0HP,5.3L,8,Cylinder,Engine,Gasoline,Fuel,None,None
36180,132.0HP,1.8L,4,Cylinder,Engine,Gasoline,Fuel,None,None
36181,450.0HP,3.5L,V6,Cylinder,Engine,Gasoline,Fuel,None,None


In [299]:
split_engine = df['engine'].str.split(' ',expand = True)

split_engine.loc[split_engine[0].str.contains('HP'), 'horsepower'] = split_engine.loc[split_engine[0].str.contains('HP'), 0].str.replace('HP', "").astype(float)
split_engine.loc[(split_engine[0].str.contains('HP')) & (split_engine[1].str.contains('L')), 'litres'] = split_engine.loc[(split_engine[0].str.contains('HP')) & (split_engine[1].str.contains('L')), 1].str.replace('L', "").astype(float)
split_engine.loc[split_engine[0].str.contains('HP') & (split_engine[3] == 'Cylinder'), 'cylinders'] = split_engine.loc[split_engine[0].str.contains('HP') & (split_engine[3] == 'Cylinder'), 2].str.replace('V', "").astype(int)
split_engine.loc[split_engine[0].str.contains('HP') & (split_engine[3] == '6'), 'cylinders'] = 6

split_engine.loc[(~split_engine[0].str.contains('HP')) & (split_engine[0].str.contains('L')), 'litres'] = pd.to_numeric(split_engine.loc[(~split_engine[0].str.contains('HP')) & (split_engine[0].str.contains('L')), 0].str.replace('L', ''), errors = 'coerce')
split_engine.loc[(~split_engine[0].str.contains('HP')) & (split_engine[0].str.contains('V')) & (split_engine['cylinders'].isnull()), 'cylinders'] = abs(pd.to_numeric(split_engine.loc[(~split_engine[0].str.contains('HP')) & (split_engine[0].str.contains('V'))][0].str.replace('V', ''), errors = 'coerce'))


split_engine.loc[(~split_engine[0].str.contains('HP')) & (split_engine[1].str.contains('L')), 'litres'] = pd.to_numeric(split_engine.loc[(~split_engine[0].str.contains('HP')) & (split_engine[1].str.contains('L')), 1].str.replace('L', ''), errors = 'coerce')
split_engine.loc[(~split_engine[0].str.contains('HP')) & (split_engine[1].str.contains('I')), 'cylinders'] = abs(pd.to_numeric(split_engine.loc[(~split_engine[0].str.contains('HP')) & (split_engine[1].str.contains('I'))][1].str.replace('I', '')))
split_engine.loc[(~split_engine[0].str.contains('HP')) & (split_engine[1].str.contains('H')), 'cylinders'] = abs(pd.to_numeric(split_engine.loc[(~split_engine[0].str.contains('HP')) & (split_engine[1].str.contains('H'))][1].str.replace('H', ''), errors = 'coerce'))
split_engine.loc[(~split_engine[0].str.contains('HP')) & (split_engine[1].str.contains('W')), 'cylinders'] = abs(pd.to_numeric(split_engine.loc[(~split_engine[0].str.contains('HP')) & (split_engine[1].str.contains('W'))][1].str.replace('W', ''), errors = 'coerce'))
split_engine.loc[(~split_engine[0].str.contains('HP')) & (split_engine[1].str.contains('V')), 'cylinders'] = abs(pd.to_numeric(split_engine.loc[(~split_engine[0].str.contains('HP')) & (split_engine[1].str.contains('V'))][1].str.replace('V', ''), errors = 'coerce'))
split_engine.loc[(~split_engine[0].str.contains('HP')) & (split_engine[1].isin([str(x) for x in list(range(20))])), 'cylinders'] = split_engine.loc[(~split_engine[0].str.contains('HP')) & (split_engine[1].isin([str(x) for x in list(range(20))]))][1].astype(float)

split_engine.loc[(~split_engine[0].str.contains('HP')) & (split_engine[2].str.contains('V')) & (split_engine['cylinders'].isnull()), 'cylinders'] = abs(pd.to_numeric(split_engine.loc[(~split_engine[0].str.contains('HP')) & (split_engine[2].str.contains('V')) & (split_engine['cylinders'].isnull())][2].str.replace('V', '')))
split_engine.loc[(~split_engine[0].str.contains('HP')) & (split_engine[2].str.contains('I')) & (split_engine['cylinders'].isnull()), 'cylinders'] = abs(pd.to_numeric(split_engine.loc[(~split_engine[0].str.contains('HP')) & (split_engine[2].str.contains('I')) & (split_engine['cylinders'].isnull())][2].str.replace('I', ''), errors = 'coerce'))
split_engine.loc[(~split_engine[0].str.contains('HP')) & (split_engine[2].str.contains('L')) & (split_engine['cylinders'].isnull()), 'litres'] = split_engine.loc[(~split_engine[0].str.contains('HP')) & (split_engine[2].str.contains('L')) & (split_engine['cylinders'].isnull())][2].str.replace('L', '')

split_engine.loc[(~split_engine[0].str.contains('HP')) & (split_engine[3].str.contains('V')) & (split_engine['cylinders'].isnull()), 'cylinders'] = abs(pd.to_numeric(split_engine.loc[(~split_engine[0].str.contains('HP')) & (split_engine[3].str.contains('V')) & (split_engine['cylinders'].isnull())][3].str.replace('V', ''), errors = 'coerce'))

split_engine.loc[(~split_engine[0].str.contains('HP')) & (split_engine[4].str.contains('I')) & (split_engine['cylinders'].isnull()), 'cylinders'] = abs(pd.to_numeric(split_engine.loc[(~split_engine[0].str.contains('HP')) & (split_engine[4].str.contains('I')) & (split_engine['cylinders'].isnull())][4].str.replace('I', ''), errors = 'coerce'))
split_engine.loc[(~split_engine[0].str.contains('HP')) & (split_engine[4].str.contains('V')) & (split_engine['cylinders'].isnull()), 'cylinders'] = abs(pd.to_numeric(split_engine.loc[(~split_engine[0].str.contains('HP')) & (split_engine[4].str.contains('V')) & (split_engine['cylinders'].isnull())][4].str.replace('V', ''), errors = 'coerce'))

split_engine.loc[(split_engine[1] == 'Liter') & (split_engine['litres'].isnull()), 'litres'] = split_engine.loc[(split_engine[1] == 'Liter') & (split_engine['litres'].isnull())][0].astype(float)

df = pd.concat([df, split_engine[['horsepower', 'litres', 'cylinders']]], axis = 1)

C:\Users\avira\AppData\Local\Temp\ipykernel_24528\1637669894.py:21: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  split_engine.loc[(~split_engine[0].str.contains('HP')) & (split_engine[2].str.contains('L')) & (split_engine['cylinders'].isnull()), 'litres'] = split_engine.loc[(~split_engine[0].str.contains('HP')) & (split_engine[2].str.contains('L')) & (split_engine['cylinders'].isnull())][2].str.replace('L', '')


In [300]:
df.loc[df['transmission'].str.contains('automatic|Automatic|A/T|AT|Auto|CVT'), 'tsm'] = 'Automatic'
df.loc[df['transmission'].str.contains('M/T|Manual|Mt'), 'tsm'] = 'Manual'
df.loc[df['transmission'].str.contains('Dual Shift Mode'), 'tsm'] = 'DCT'
df['tsm'].fillna('Other', inplace = True)

C:\Users\avira\AppData\Local\Temp\ipykernel_24528\2197479763.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['tsm'].fillna('Other', inplace = True)


In [301]:
df.loc[df['ext_col'].str.contains('Black'), 'exterior_color'] = 'Black'
df.loc[df['ext_col'].str.contains('White'), 'exterior_color'] = 'White'
df['exterior_color'].fillna('Other', inplace = True)

C:\Users\avira\AppData\Local\Temp\ipykernel_24528\1977337176.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['exterior_color'].fillna('Other', inplace = True)


In [302]:
df.loc[df['int_col'].str.contains('Black'), 'interior_color'] = 'Black'
df.loc[df['int_col'].str.contains('White'), 'interior_color'] = 'White'
df['interior_color'].fillna('Other', inplace = True)

C:\Users\avira\AppData\Local\Temp\ipykernel_24528\3787901577.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['interior_color'].fillna('Other', inplace = True)


In [303]:
df['accident'] = df['accident'].replace('At least 1 accident or damage reported',1)
df.loc[df['accident'] != 1, 'accident'] = 0

df['clean_title'] = df['clean_title'].replace('Yes', 1).fillna(0)

C:\Users\avira\AppData\Local\Temp\ipykernel_24528\3780501552.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['clean_title'] = df['clean_title'].replace('Yes', 1).fillna(0)


In [304]:
bin_edges = range(1970, 2031, 5)
bin_labels = [f'{start}-{start+9}' for start in bin_edges[:-1]]
df['model_year_bins'] = pd.cut(df['model_year'], bins=bin_edges, labels=bin_labels, right=False)

In [305]:
categorical_columns = df.select_dtypes(include=['object', 'category']).columns


In [306]:
df['litres'] = df['litres'].astype(float)

In [307]:
df = pd.merge(df, df.groupby(['litres', 'cylinders'])['horsepower'].mean().reset_index().rename({'horsepower': 'hp_mean'}, axis = 1),
         'left', ['litres', 'cylinders'])
df.loc[df['horsepower'].isnull() & df['hp_mean'].notnull(), 'horsepower'] = round(df.loc[df['horsepower'].isnull() & df['hp_mean'].notnull(), 'hp_mean'])

In [283]:
df = df[df['price']<=200000].copy().reset_index(drop = True)

In [308]:
from sklearn.preprocessing import OrdinalEncoder
OHE = OrdinalEncoder()
df[categorical_columns] = OHE.fit_transform(df[categorical_columns])

In [285]:
X = df.copy()
y = df['price']

In [286]:
X.drop('price',axis=1,inplace=True)
# X.drop('id',axis=1,inplace=True)

In [287]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [288]:
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor

# Define the parameter grid for GridSearchCV
param_grid = {
    'n_estimators': [100, 500, 1000],
    'max_depth': [3, 5, 7],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Create the Random Forest Regression model
rf_model = RandomForestRegressor(random_state=42)

# Create GridSearchCV object with cross-validation
grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)

# Fit the GridSearchCV object to the training data
grid_search.fit(X_train, y_train)

# Get the best parameters and best estimator
best_params = grid_search.best_params_
best_estimator = grid_search.best_estimator_

# Make predictions on training and testing data using the best estimator
y_train_pred_rf = best_estimator.predict(X_train)
y_test_pred_rf = best_estimator.predict(X_test)

# Calculate R-squared and mean squared error for evaluation
rf_r2_train = r2_score(y_train, y_train_pred_rf)
rf_r2_test = r2_score(y_test, y_test_pred_rf)
rf_mse_train = mean_squared_error(y_train, y_train_pred_rf)
rf_mse_test = mean_squared_error(y_test, y_test_pred_rf)

print("Best Parameters:", best_params)
print(f"\nRandom Forest Regression (Bagging):")
print(f"Training R-squared: {rf_r2_train:.4f}, Training MSE: {rf_mse_train:.4f}")
print(f"Testing R-squared: {rf_r2_test:.4f}, Testing MSE: {rf_mse_test:.4f}")

Best Parameters: {'max_depth': 7, 'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 1000}

Random Forest Regression (Bagging):
Training R-squared: 0.5550, Training MSE: 343828610.2519
Testing R-squared: 0.5271, Testing MSE: 336140972.0155


In [309]:
predictions = best_estimator.predict(df)

In [310]:
submission_df = pd.DataFrame({'id': df['id'], 'price': predictions})
submission_df.to_csv('submission.csv', index=False)